<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/Demucs_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demucs による音源分離

論文<br>
https://arxiv.org/abs/2111.03600<br>
<br>
GitHub<br>
https://github.com/facebookresearch/demucs<br>
<br>

# ランタイムの設定
「ランタイム」→「ランタイムのタイプを変更」→「ハードウェアアクセラレータ」をGPUに変更

# 実行方法
「ランタイム」→「すべてのセルを実行」を選択

# 環境セットアップ

In [ ]:
!nvidia-smi

## ライブラリのインストール

In [ ]:
!pip install demucs
!pip install yt-dlp moviepy
!pip install imageio #==2.4.1
!pip install pydub

## ライブラリのインポート

In [ ]:
from yt_dlp import YoutubeDL

from moviepy.video.fx.resize import resize
from moviepy.editor import VideoFileClip, AudioFileClip, ImageSequenceClip, CompositeAudioClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import IPython.display as ipd

# テスト動画のセットアップ

In [ ]:
video_url = 'https://www.youtube.com/watch?v=D6DVTLvOupE' #@param {type:"string"}

#@markdown 動画の切り抜き範囲(秒)を指定してください。\
#@markdown 30秒以上の場合OOM発生の可能性が高いため注意
start_sec =  0#@param {type:"integer"}
end_sec =  30#@param {type:"integer"}

(start_pt, end_pt) = (start_sec, end_sec)

## ダウンロード

In [ ]:
!mkdir -p videos audios

download_resolution = 360
full_video_path = 'videos/full_video.mp4'
input_clip_path = 'videos/input_clip.mp4'
input_audio_path = 'audios/input_clip.wav'

# 動画ダウンロード
ydl_opts = {'format': f'best[height<={download_resolution}]', 'overwrites': True, 'outtmpl': full_video_path}
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

## clip

In [ ]:
# 指定区間切り抜き
with VideoFileClip(full_video_path) as video:
    subclip = video.subclip(start_pt, end_pt)
    subclip.write_videofile(input_clip_path)

## 音声ファイル保存

In [ ]:
# 音声抽出
clip = VideoFileClip(input_clip_path)
clip.audio.write_audiofile(input_audio_path, codec='pcm_s16le')

ipd.Audio(input_audio_path, rate=44100)

# 音源分離

In [ ]:
!python3 -m demucs {input_audio_path}

## パートごとの再生

### ヴォーカル

In [ ]:
vocal_wav = 'separated/htdemucs/input_clip/vocals.wav'
ipd.Audio(vocal_wav, rate=44100)

### ドラム

In [ ]:
drums_wav = 'separated/htdemucs/input_clip/drums.wav'
ipd.Audio(drums_wav, rate=44100)

### ベース

In [ ]:
bass_wav = 'separated/htdemucs/input_clip/bass.wav'
ipd.Audio(bass_wav, rate=44100)

### その他

In [ ]:
other_wav = 'separated/htdemucs/input_clip/other.wav'
ipd.Audio(other_wav, rate=44100)

## カラオケ動画を作成

### ボーカル以外の抽出

In [ ]:
!python3 -m demucs --two-stems=vocals {input_audio_path}

### 確認

In [ ]:
karaoke_wav = 'separated/htdemucs/input_clip/no_vocals.wav'
ipd.Audio(karaoke_wav, rate=44100)

### 映像と音声の合成

In [ ]:
karaoke_video = 'karaoke.mp4'

videoclip = VideoFileClip('videos/input_clip.mp4')
audioclip = AudioFileClip('separated/htdemucs/input_clip/no_vocals.wav')

karaokeclip = videoclip.set_audio(audioclip)
karaokeclip.write_videofile(karaoke_video, codec='libx264', audio_codec='aac', temp_audiofile='temp-audio.m4a', remove_temp=True)

### 再生

In [ ]:
# mp4動画の再生
from IPython.display import HTML
from base64 import b64encode

mp4 = open('karaoke.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="50%" height="50%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")